In [1]:
WFH = False

In [2]:
if WFH:
  pass
else:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import shutil
import pandas as pd
import ast

import os

In [9]:
def get_image_paths(folder_path, extensions=('jpg', 'jpeg', 'png', 'bmp')):
    image_paths = []

    # List all files in the folder
    all_files = os.listdir(folder_path)

    # Filter files based on extensions
    image_paths = [os.path.join(folder_path, file) for file in all_files if file.lower().endswith(extensions)]

    return image_paths

In [10]:
image_data = get_image_paths("/Datasets/Grid Dataset/Images")

In [11]:
df = pd.read_csv("/Datasets/Grid Dataset/dataset.csv")
df.head()

,Image,Coordinates
0,CC_1.jpg,"[[473.28, 500.48], [480, 491.84], [490.88, 467..."
1,CC_2.jpg,"[[476.16, 497.6], [483.84, 475.84], [489.6, 44..."
2,CC_3.jpg,"[[449.28, 286.4], [450.189, 271.81], [449.28, ..."
3,CC_4.jpg,"[[450.917, 257.569], [449.086, 241.292], [445...."
4,CC_5.jpg,"[[448.296, 312.682], [449.583, 301.442], [451...."


In [12]:
len(df)

3373

In [13]:
test_path = "/Datasets/Grid Dataset/Grid-Applied Images"
os.makedirs(test_path, exist_ok = True)

enlargement_factor = 0.75
y_shift_top = 0.2
y_shift_bottom = 0.2

for i in range(len(image_data)):
    img_path = image_data[i]
    img_name = img_path.split("/")[-1]
    coord = df.loc[df['Image'] == img_name, 'Coordinates'].values[0]
    coord = ast.literal_eval(coord)  # Convert from string to list
    x_coords, y_coords = zip(*coord)

    # Vertical lines
    min_y, max_y = min(y_coords), max(y_coords)
    top_edge = min_y + ((max_y-min_y) * y_shift_top)
    bottom_edge = max_y - ((max_y-min_y) * y_shift_bottom)

    # Side contour lines
    plot_coordinates = np.array(coord)
    centre = np.mean(plot_coordinates, axis=0)

    translated_coords = plot_coordinates - centre
    scaled_coords = translated_coords * enlargement_factor
    enlarged_coords = scaled_coords + centre
    enlarged_coordinates = [tuple(coord) for coord in enlarged_coords]

    enlarged_x_coords, enlarged_y_coords = zip(*enlarged_coordinates)

    # Generate background contour
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    plt.figure(i)
    plt.imshow(image_rgb)
    plt.plot(enlarged_x_coords, enlarged_y_coords, linestyle='-', color='r')  # Red markers and lines
    plt.axhline(y=top_edge, color='g', linestyle='-')
    plt.axhline(y=bottom_edge, color='b', linestyle='-')
    plt.title(f'{img_name}')
    plt.xlabel('X-coordinate')
    plt.ylabel('Y-coordinate')

    background_contour_path = test_path + "/" + img_name
    plt.savefig(background_contour_path)
    plt.close()